<a href="https://colab.research.google.com/github/Nabladx/GoogleColabs/blob/main/OCSVMR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install natasha
!pip install flair
!pip install -U sentence-transformers
!pip install django-model-utils

     |████████████████████████████████| 34.4 MB 99 kB/s 
     |████████████████████████████████| 41 kB 43 kB/s 
     |████████████████████████████████| 55 kB 4.6 MB/s 
     |████████████████████████████████| 49 kB 3.5 MB/s 
     |████████████████████████████████| 8.2 MB 6.6 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=1ca1770743924d1e4a4f0b13a467999f027dfe725d9795adea4b3dee50e0420d
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
Successfully built intervaltree
  Attempting uninstall: intervaltree
    Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0
     |████████████████████████████████| 401 kB 23.6 MB/s 
     |████████████████████████████████| 1.6 MB 53.5 MB/s 
     |████████████████████████████████| 4.0 MB 61.7 MB/s 
     |████████████████████████████████| 1.2 MB 50.3 MB/s 
     |

In [3]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    LOC,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,
    Doc
)
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from django import forms
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

file1 = open('/content/drive/MyDrive/datasets/OneClassSVMRussia/train_data/Арцыбашев М. П..txt')
file2 = open('/content/drive/MyDrive/datasets/OneClassSVMRussia/test_data/Арцыбашев М. П..txt')

train_data = file1.read()
test_data = file2.read()

doc1 = Doc(train_data)
doc2 = Doc(test_data)

doc1.segment(segmenter)
doc2.segment(segmenter)
doc1.tag_morph(morph_tagger)
doc2.tag_morph(morph_tagger)
doc1.parse_syntax(syntax_parser)
doc2.parse_syntax(syntax_parser)
doc1.tag_ner(ner_tagger)
doc2.tag_ner(ner_tagger)

noun_train = 0
adv_train = 0
adp_train = 0
adj_train = 0
pron_train = 0
num_train = 0
cconj_train = 0
aux_train = 0 
noun_test = 0
adv_test = 0
adp_test = 0
adj_test = 0
pron_test = 0
num_test = 0
cconj_test = 0
aux_test = 0 

train_words = 10000
for i in range (1,train_words):
    if doc1.tokens[i].pos == 'NOUN': 
        noun_train+=1
    if doc1.tokens[i].pos == 'ADV': 
        adv_train+=1
    if doc1.tokens[i].pos == 'ADP': 
        adp_train+=1  
    if doc1.tokens[i].pos == 'ADJ':      
        adj_train+=1
    if doc1.tokens[i].pos == 'PRON':      
        pron_train+=1
    if doc1.tokens[i].pos == 'NUM':
        num_train+=1
    if doc1.tokens[i].pos == 'CCONJ':
        cconj_train+=1
    if doc1.tokens[i].pos == 'AUX':
        aux_train+=1
        
test_words = 25
for i in range (1,test_words):
    if doc2.tokens[i].pos == 'NOUN': 
        noun_test+=1
    if doc2.tokens[i].pos == 'ADV': 
        adv_test+=1
    if doc2.tokens[i].pos == 'ADP': 
        adp_test+=1 
    if doc2.tokens[i].pos == 'ADJ':      
        adj_test+=1
    if doc2.tokens[i].pos == 'PRON':      
        pron_test+=1
    if doc2.tokens[i].pos == 'NUM':
        num_test+=1
    if doc2.tokens[i].pos == 'CCONJ':
        cconj_test+=1
    if doc2.tokens[i].pos == 'AUX':
        aux_test+=1
        
NOUN_train = noun_train / train_words
ADV_train = adv_train / train_words 
ADP_train = adp_train / train_words 
ADJ_train = adj_train / train_words 
PRON_train = pron_train / train_words 
NUM_train = num_train / train_words 
CCONJ_train = cconj_train / train_words 
AUX_train = aux_train / train_words 

NOUN_test = noun_test / test_words 
ADV_test = adv_test / test_words 
ADP_test = adp_test / test_words 
ADJ_test = adj_test / test_words 
PRON_test = pron_test / test_words 
NUM_test = num_test / test_words 
CCONJ_test = cconj_test / test_words 
AUX_test = aux_test / test_words 

NOUN_train = [NOUN_train] 
ADV_train = [ADV_train] 
ADP_train = [ADP_train] 
ADJ_train = [ADJ_train] 
PRON_train = [PRON_train] 
NUM_train = [NUM_train] 
CCONJ_train = [CCONJ_train] 
AUX_train = [AUX_train]
category_train = [NOUN_train, ADV_train, ADP_train, ADJ_train, 
                  PRON_train, NUM_train, CCONJ_train, AUX_train]

NOUN_test = [NOUN_test] 
ADV_test = [ADV_test] 
ADP_test = [ADP_test] 
ADJ_test = [ADJ_test] 
PRON_test = [PRON_test] 
NUM_test = [NUM_test] 
CCONJ_test = [CCONJ_test] 
AUX_test = [AUX_test]
category_test = [NOUN_test, ADV_test, ADP_test, ADJ_test,
                PRON_test, NUM_test, CCONJ_test, AUX_test]

clm = OneClassSVM(gamma='auto').fit(category_train)
clm.predict(category_train)
clf = OneClassSVM(gamma='auto').fit(category_test)
clf.predict(category_test)

pred_train = clf.predict(category_train)
pred_test = clf.predict(category_test)

results = confusion_matrix(pred_train, pred_test) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(pred_train, pred_test)) 
print('Report : ')
print(classification_report(pred_train, pred_test)) 

Confusion Matrix :
[[4 1]
 [1 2]]
Accuracy Score : 0.75
Report : 
              precision    recall  f1-score   support

          -1       0.80      0.80      0.80         5
           1       0.67      0.67      0.67         3

    accuracy                           0.75         8
   macro avg       0.73      0.73      0.73         8
weighted avg       0.75      0.75      0.75         8

